In [2]:
import ete3
import os
import re
import random
import numpy as np
import pandas as pd
import seaborn as sns
from itertools import combinations, product
from matplotlib import pyplot as plt
import matplotlib

class cd:
    """
    Context manager for changing the current working directory
    """
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

%cd /work/site_rate/hug_et_al

/work/site_rate/hug_et_al


In [7]:
cmap = plt.cm.rainbow
norm = matplotlib.colors.Normalize(vmin=0, vmax=8)

taxa = ['Euryarchaeota',   'DPANN',        'Thaumarchaeota',
        'Asgardarchaeota', 'Aigarchaeota', 'Crenarchaeota',
        'Korarchaeota',    'Eukaryota',    'Nanohaloarchaeota',]

nameFaces = {}

colors = '#6E9D34 #8E8CD9 #B4423C #3D7AB8 #3DB8B8 #6038BC #FFBC1F #BD008E'.split()
for position, taxon in  enumerate(taxa):
    if position >= len(colors):
        position -= len(colors)
    nameFaces[taxon] = ete3.AttrFace("taxon",
                                     fsize=10,
                                     fgcolor=colors[position]
                                    )
#colors = ['#%02x%02x%02x' % cmap(norm(index), bytes=8)[:3]
#          for index in range(0,13)]
#for position, taxon in enumerate(taxa):
#    nameFaces[taxon] = ete3.AttrFace("taxon",
#                                     fsize=10,
#                                     fgcolor='#%02x%02x%02x' % cmap(norm(position), bytes=8)[:3]
#                                    )
nameFaces['Unknow']= ete3.AttrFace(
    "name",
    fsize=10,
    fgcolor='#000000'
)

In [8]:
extra_taxonomic_members = {}
extra_taxonomic_members['Euryarchaeota'] = 'Archaea_uncultured_halophilic_archaeon_J07ABHX64\
                                            Archaea_uncultured_halophilic_archaeon_J07ABHX67_version_2\
                                            Archaea_uncultured_archaeon_J07ABHN6_version_2\
                                            Archaea_Thaumarchaeota_unclassified_Thaumarchaeota_Marine_Group_I_Marine_Group_III_euryarchaeote_sp._SCGC_AAA288_E19\
                                            Archaea_Hadesarchaea_archaeon_YNP_45\
                                            Archaea_Hadesarchaea_archaeon_YNP_N21\
                                            Archaea_CP_Z7ME43_WOR_bin_SMTZ_36\
                                            Archaea_CP_Z7ME43_WOR_bin_DG_70'.split()


In [3]:
tmp = ete3.Tree('/work/site_rate/hug_et_al/ribosomal_concat.fasta.treefile',
                 format=1)
bacteria     = []
archaea      = []
eukarya      = []
for leaf in tmp.get_leaf_names():
    if leaf.startswith('Bacteria'):
        bacteria.append(leaf)
    elif leaf.startswith('Archaea'):
        archaea.append(leaf)
    elif leaf.startswith('Eukaryota'):
        eukarya.append(leaf)

tmp.set_outgroup(eukarya[0])
bacteria_ancestor = tmp.get_common_ancestor(bacteria)
tmp.set_outgroup(bacteria_ancestor)

In [4]:
lg_trees   = ['../whole_alignment_FU.treefile',
              'no_fast_evolving_sites.aln.treefile',
              'no_slow_evolving_sites.aln.treefile', 
              'just_mid_evolving_sites.aln.treefile']
c60_trees  = ['C60_ribosomal_concat.fasta.treefile',
              'C60_no_fast_evolving_sites.aln.treefile',
              'C60_no_slow_evolving_sites.aln.treefile',
              'C60_just_mid_evolving_sites.aln.treefile']
lg4x_trees = ['LG4X_ribosomal_concat.fasta.treefile',
              'LG4X_no_fast_evolving_sites.aln.treefile',
              'LG4X_no_slow_evolving_sites.aln.treefile',
              'LG4X_just_mid_evolving_sites.aln.treefile']
no_gappy =   ['whole_alignment_no_gappy_sites.aln_LG.treefile',
              'C60_whole_alignment_no_gappy_sites.aln.treefile',
              'LG4X_whole_alignment_no_gappy_sites.aln.treefile']

In [5]:
with cd('best_phylogenies/'):
    print('LG')
    for filename in lg_trees:
        if not filename:
            continue

        tree = ete3.Tree(filename,
                         format=0)
        tree.set_outgroup(eukarya[0])
        bacteria_ancestor = tree.get_common_ancestor(bacteria)
        tree.set_outgroup(bacteria_ancestor)
        tree = tree.get_common_ancestor(archaea+eukarya).copy()

        print(
            '\t',
            filename.replace('../', '').split('.')[0],
            len(
                [node for node in tree.traverse() if not node.is_leaf() and node.support > 95]
            )
        )

    print('C60')
    for filename in c60_trees:
        if not filename:
            continue

        tree = ete3.Tree(filename,
                         format=0)
        tree.set_outgroup(eukarya[0])
        bacteria_ancestor = tree.get_common_ancestor(bacteria)
        tree.set_outgroup(bacteria_ancestor)
        tree = tree.get_common_ancestor(archaea+eukarya).copy()

        print(
            '\t',
            filename.replace('../', '').split('.')[0],
            len(
                [node for node in tree.traverse() if not node.is_leaf() and node.support > 95]
            )
        )

    print('LG4X')
    for filename in lg4x_trees:
        if not filename:
            continue

        tree = ete3.Tree(filename,
                         format=0)
        tree.set_outgroup(eukarya[0])
        bacteria_ancestor = tree.get_common_ancestor(bacteria)
        tree.set_outgroup(bacteria_ancestor)
        tree = tree.get_common_ancestor(archaea+eukarya).copy()

        print(
            '\t',
            filename.replace('../', '').split('.')[0],
            len(
                [node for node in tree.traverse() if not node.is_leaf() and node.support > 95]
            )
        )
    print('no gap-rich')
    for filename in no_gappy:
        if not filename:
            continue

        tree = ete3.Tree(filename,
                         format=0)
        tree.set_outgroup(eukarya[0])
        bacteria_ancestor = tree.get_common_ancestor(bacteria)
        tree.set_outgroup(bacteria_ancestor)
        tree = tree.get_common_ancestor(archaea+eukarya).copy()

        print(
            '\t',
            filename.replace('../', '').split('.')[0],
            len(
                [node for node in tree.traverse() if not node.is_leaf() and node.support > 95]
            )
        )

LG
	 whole_alignment_FU 279
	 no_fast_evolving_sites 266
	 no_slow_evolving_sites 267
	 just_mid_evolving_sites 261
C60
	 C60_ribosomal_concat 277
	 C60_no_fast_evolving_sites 270
	 C60_no_slow_evolving_sites 265
	 C60_just_mid_evolving_sites 270
LG4X
	 LG4X_ribosomal_concat 280
	 LG4X_no_fast_evolving_sites 274
	 LG4X_no_slow_evolving_sites 260
	 LG4X_just_mid_evolving_sites 233
no gap-rich
	 whole_alignment_no_gappy_sites 275
	 C60_whole_alignment_no_gappy_sites 274
	 LG4X_whole_alignment_no_gappy_sites 280


In [10]:
with cd('best_phylogenies/'):
    for filename in no_gappy:
        if not filename:
            continue

        tree = ete3.Tree(filename,
                         format=0)
        tree.set_outgroup(eukarya[0])
        bacteria_ancestor = tree.get_common_ancestor(bacteria)
        tree.set_outgroup(bacteria_ancestor)

        subtree = tree.get_common_ancestor(archaea+eukarya).copy()
        for leaf in subtree.get_leaves():
            leaf.name = leaf.name.replace("'", '')
            for taxon in taxa:
                if taxon in leaf.name:
                    leaf.add_feature('taxon', taxon)
            if 'taxon' not in leaf.features:
                if 'Lokiarchaeota' in leaf.name or 'Lokiarchaeaota' in leaf.name  or 'Thorarchaeota' in leaf.name:
                    leaf.add_feature('taxon', 'Asgardarchaeota')
                elif 'Nanoarchaeota' in leaf.name:
                    leaf.add_feature('taxon', 'DPANN')
                else:
                    leaf.add_feature('taxon', 'Unknow')

            if leaf.name in extra_taxonomic_members['Euryarchaeota']:
                leaf.taxon = 'Euryarchaeota'

        node_content = subtree.get_cached_content()

        nodeStyle = ete3.NodeStyle()

        subtree.ladderize()
        for node, descendants in node_content.items():
            node.img_style['size'] = 0
            node.img_style["vt_line_width"] = 1
            node.img_style["hz_line_width"] = 1

            if len(descendants) == 1:
        #        node.add_face(nameFaces[node.taxon], 1)
                pass
            else:
                descendant_taxa = set([leaf.taxon for leaf in descendants])
        #        node.support = float(node.name.split('/')[1])
                if node.support >= 95:
                    node.img_style['size']    = 3
                    node.img_style['fgcolor'] = '#B4423C'

                if len(descendant_taxa) > 1:
                    continue

                taxon = descendant_taxa.pop()

                if taxon == 'Unknow':
                    continue

                node.add_feature('taxon', taxon)
                node.add_face(
                    ete3.AttrFace(
                        "taxon",
                        fsize=15+len(descendants)*0,
                        fgcolor=nameFaces[taxon].fgcolor
                    ),
                    1
                )
                node.img_style['draw_descendants'] = False

        treeStyle                   = ete3.TreeStyle()
        treeStyle.branch_vertical_margin = 1
        treeStyle.show_leaf_name    = False

        outname = filename.replace('../', '').split('.')[0]
        figure = subtree.render(
            file_name='/Users/thiberio/Dropbox (MIT)/site_rates-paper/figures/collapsed_topologies/%s.pdf'
            % outname,
            tree_style=treeStyle,
            w=2000,
            dpi=600)